In [27]:
import numpy as np
import plotly.graph_objects as go

In [28]:
np.random.seed(42)  # for reproducibility
N = 10_000          # number of samples

# Target: N(0, 1)
mu_pi = 0.0
sigma_pi = 1.0

# Proposal: N(0, 3^2)
mu_q = 0.0
sigma_q = 3.0

# Function of interest
def f(theta):
    return theta**2

In [29]:
### Density definitions ###

def normal_pdf(x, mu, sigma):
    coeff = 1.0 / (np.sqrt(2.0 * np.pi) * sigma)
    return coeff * np.exp(-0.5 * ((x - mu) / sigma)**2)

def pi_pdf(theta):
    # Target density π(θ)
    return normal_pdf(theta, mu_pi, sigma_pi)

def q_pdf(theta):
    # Proposal density q(θ)
    return normal_pdf(theta, mu_q, sigma_q)

In [30]:
### Importance Sampling ###

# Sample from proposal q
theta_samples = np.random.normal(loc=mu_q, scale=sigma_q, size=N)

# Compute unnormalized weights: w(θ) = π(θ) / q(θ)
weights = pi_pdf(theta_samples) / q_pdf(theta_samples)

# Estimate of Z (normalizing constant; here it should be 1)
Z_hat = np.mean(weights)

# Normalized weights
weights_normalized = weights / np.sum(weights)

# Estimate of the integral I = E_pi[f(θ)]
I_hat = np.sum(weights_normalized * f(theta_samples))

# True value for comparison (E_pi[θ^2] = 1 for N(0,1))
I_true = 1.0

print("Number of samples N         :", N)
print("Estimated Z_hat             :", Z_hat)
print("Estimated I_hat (IS)        :", I_hat)
print("True value I_true           :", I_true)
print("Absolute error |I_hat - 1|  :", abs(I_hat - I_true))

Number of samples N         : 10000
Estimated Z_hat             : 1.0012440081007572
Estimated I_hat (IS)        : 0.998644322389725
True value I_true           : 1.0
Absolute error |I_hat - 1|  : 0.0013556776102749968


In [31]:
print("Number of samples N         :", N)
print("Estimated Z_hat             :", Z_hat)
print("Estimated I_hat (IS)        :", I_hat)
print("True value I_true           :", I_true)
print("Absolute error |I_hat - 1|  :", abs(I_hat - I_true))

# Grid for plotting densities
x = np.linspace(-10, 10, 1000)
pi_vals = pi_pdf(x)
q_vals = q_pdf(x)

### Plot 1: target vs proposal ###

fig1 = go.Figure()

fig1.add_trace(
    go.Scatter(
        x=x,
        y=pi_vals,
        mode="lines",
        name="Target π(θ) = N(0,1)"
    )
)

fig1.add_trace(
    go.Scatter(
        x=x,
        y=q_vals,
        mode="lines",
        name="Proposal q(θ) = N(0, 3²)",
        line=dict(dash="dash")
    )
)

fig1.update_layout(
    title="Target vs Proposal",
    xaxis_title="θ",
    yaxis_title="Density",
    legend=dict(x=0.02, y=0.98),
)

fig1.show()

### Plot 2: Weighted histogram (manual) vs target ###

hist_vals, bin_edges = np.histogram(
    theta_samples,
    bins=50,
    weights=weights_normalized,
    density=True
)

bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

fig2 = go.Figure()

# Weighted histogram
fig2.add_trace(
    go.Bar(
        x=bin_centers,
        y=hist_vals,
        name="Weighted samples (IS)",
        opacity=0.7
    )
)

# Overlay target density
fig2.add_trace(
    go.Scatter(
        x=x,
        y=pi_vals,
        mode="lines",
        name="Target π(θ)"
    )
)

fig2.update_layout(
    title="Approximating the Target Density with Importance Sampling",
    xaxis_title="θ",
    yaxis_title="Density",
    barmode="overlay"
)

fig2.show()

### Plot 3: Distribution of importance weights ###

fig3 = go.Figure()

fig3.add_trace(
    go.Histogram(
        x=weights,
        histnorm="probability density",
        nbinsx=50
    )
)

fig3.update_layout(
    title="Distribution of Weights w(θ) = π(θ) / q(θ)",
    xaxis_title="w(θ)",
    yaxis_title="Density"
)

fig3.show()


Number of samples N         : 10000
Estimated Z_hat             : 1.0012440081007572
Estimated I_hat (IS)        : 0.998644322389725
True value I_true           : 1.0
Absolute error |I_hat - 1|  : 0.0013556776102749968
